In [53]:
# Basic
import  torch
from   torch import nn
from d2l import torch as d2l

In [63]:
def corr2d(X, K):
    h, w = K.shape # height and width of the kernel
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [55]:
# Test
X = torch.tensor([[0.0, 1.0, 2.0], 
                  [3.0, 4.0, 5.0], 
                  [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0],
                  [2.0, 3.0]])

corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [56]:
# convolution
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forwad(self, x):
        return corr2d(x, self.weight) + self.bias

In [57]:
# Edge detection
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [64]:
K = torch.tensor([[1.0, -1.0]])
Y = corr2d(X, K)
Y

tensor([], size=(1, 0))

In [65]:
# Vertical edges only
Y = corr2d(X.t(), K)
Y

RuntimeError: t() expects a tensor with <= 2 dimensions, but self is 4D

In [66]:
# Specifying conv kernel is difficult, could it be a process of learning?
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)
# batch_size, channel, height, width
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2

for i in range(10):
    Y_hat = conv2d(X)
    loss = (Y - Y_hat)**2
    conv2d.zero_grad()      # clear former gradient
    loss.sum().backward()      # excute backward process

    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    print(f'epoch {i+1}, loss {loss.sum():.3f}')
    

RuntimeError: shape '[1, 1, 6, 7]' is invalid for input of size 0

In [67]:
conv2d.weight.data

tensor([[[[-0.1712,  0.1016]]]])

In [68]:
def comp_conv2d(conv2d, X):
    """Compute the size of convolution layer"""
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # return ignoring the first 2 parameter, batch_size and channel
    return Y.reshape(Y.shape[2:]) 

conv2d = nn.Conv2d(1, 1, kernel_size = 3, padding = 1)
X = torch.rand(size = (8, 8))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [69]:
# Non-Square convolution kernel
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding = (2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [70]:
# Stride
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

In [71]:
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding = (0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])

In [72]:
# Multiple input and single output Channels
def corr2d_multi_in(X, K):
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

![example](/Users/kehao/CodeSpace/d2l-footprints/appendix/multiple_input_single_output.png)

In [73]:
X = torch.tensor([
                [[0.0, 1.0, 2.0], 
                 [3.0, 4.0, 5.0], 
                 [6.0, 7.0, 8.0]],

                [[1.0, 2.0, 3.0], 
                 [4.0, 5.0, 6.0], 
                 [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K),X.shape

(tensor([[ 56.,  72.],
         [104., 120.]]),
 torch.Size([2, 3, 3]))

In [74]:
# Multiple input and output
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [75]:
print(K.shape)
# channels = 2, kernel_size = (2, 2)
K = torch.stack((K, K+1, K+2), 0)
print(K.shape)

torch.Size([2, 2, 2])
torch.Size([3, 2, 2, 2])


In [77]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [78]:
# 1*1 kernel_size
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0] # defined by kernel_size
    X = X.reshape((c_i, h*w))
    K = K.reshape((c_o, c_i))

    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [79]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

In [80]:
# Pooling's target
# 1. reduct the sensitivity of convlution layer toward position
# 2. reduct the sensitivity of space sampling

def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h +1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()
    return Y

In [85]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

In [87]:
X = torch.arange(16, dtype=torch.float32).reshape(1, 1, 4, 4)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

In [88]:
# Defualtly, stride is the same with pooling size. So it's 3 in this example
pool2d = nn.MaxPool2d(3)
pool2d(X)

tensor([[[[10.]]]])

In [89]:
pool2d = nn.MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=(0, 1))
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [94]:
# multiple pooling
X = torch.arange(16, dtype=torch.float32).reshape(1, 1, 4, 4)  
X = torch.cat((X, X + 1), 1)

In [95]:
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [96]:
pool2d = nn.MaxPool2d(3, padding = 1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])